Variables

In [ ]:
SCALINGS = 4e-4
SFREQ = 500
WINDOW_SIZE = 1
SHOW = False
FMIN = 0.01
FMAX = 45
FREQ_BANDS = {
    'delta': (0.01, 4),
    'theta': (4, 8),
    'alpha1': (8, 10),
    'alpha2': (10, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}
ALL_CHANNELS=[]

Importing all packages

In [ ]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy.fft import fft
import time
import mne
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import kurtosis, zscore, ttest_ind
from mne.preprocessing import create_ecg_epochs, create_eog_epochs, read_ica
from mne.time_frequency import tfr_morlet, tfr_array_morlet, morlet, AverageTFR
from itertools import product
import pywt
from scipy.signal import hilbert
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity
import networkx as nx
import numpy as np 
import re
from scipy.cluster.hierarchy import dendrogram, linkage

matplotlib.use('TkAgg')

## Reading the EEG data

In [ ]:
# folder_path = os.getcwd()+'\\Depression-Sample-dataset-AIIMS\\'
# items = os.listdir(folder_path)
# active_or_sham_list = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]
# for active_or_sham in active_or_sham_list:
#     patient_folder_path = os.path.join(folder_path, active_or_sham)
#     items = os.listdir(patient_folder_path)
#     patients_list = [item for item in items if os.path.isdir(os.path.join(patient_folder_path, item))]
#     patients_list = ['PreetiSingh', 'Hemlata', 'VinodKumarSharma']
#     for patient in patients_list:
#         pre_post_int_folder_path = os.path.join(patient_folder_path, patient)
#         items = os.listdir(pre_post_int_folder_path)
#         pre_post_int_list = [item for item in items if os.path.isdir(os.path.join(pre_post_int_folder_path, item))]
#         for var in pre_post_int_list:
#             if var=='Pre':
#                 pre_path = os.path.join(pre_post_int_folder_path, var)
#                 file_path = pre_path + '\\' + '20230718201550_Preeti singh_22.08.23-01_Eye Close' + '.easy'
#                 break # Remove for all pre, post and intervention for a patient
#         break # Remove for all patients in active or sham
#     break # Remove for both active and sham

Data analysis

In [ ]:
# file_path = os.getcwd()+'\\Depression-Sample-dataset-AIIMS\\Active\\VinodKumarSharma\\pre\\20230829195416_VinodKumarSharma_25.9.23_01_Eye Close.easy'
def data_transformation_easy(file_path):
	df = pd.read_csv(file_path, sep='\t')
	channel_str='Channel 1:P8\
		Channel 2:T8\
		Channel 3:CP6\
		Channel 4:FC6\
		Channel 5:F8\
		Channel 6:F4\
		Channel 7:C4\
		Channel 8:P4\
		Channel 9:AF4\
		Channel 10:Fp2\
		Channel 11:Fp1\
		Channel 12:AF3\
		Channel 13:Fz\
		Channel 14:FC2\
		Channel 15:Cz\
		Channel 16:CP2\
		Channel 17:PO3\
		Channel 18:O1\
		Channel 19:Oz\
		Channel 20:O2\
		Channel 21:PO4\
		Channel 22:Pz\
		Channel 23:CP1\
		Channel 24:FC1\
		Channel 25:P3\
		Channel 26:C3\
		Channel 27:F3\
		Channel 28:F7\
		Channel 29:FC5\
		Channel 30:CP5\
		Channel 31:T7\
		Channel 32:P7'

	channel_names = re.findall(r'Channel \d+:(\w+)', channel_str)
	channel_names.append('ax')
	channel_names.append('ay')
	channel_names.append('az')
	channel_names.append('trigger')
	channel_names.append('timestamp(ms)')
	global ALL_CHANNELS
	ALL_CHANNELS = np.array(channel_names[:-5])
	df.columns=channel_names
	transposed_data=df.T
	ch_names = df.columns.tolist()[:-5]
	ch_types = ['eeg' for i in range(32)]
	info = mne.create_info(ch_names=ch_names,ch_types=ch_types, sfreq=500)
	raw = mne.io.RawArray(transposed_data.values[:-5,:]/1e9, info) # Example: 33129984 nV = 0.033129984 V = 33129.984000000004 uV
	return raw
# data_transformation_easy(file_path)

In [ ]:
ALL_CHANNELS

Set montage

In [ ]:
def set_montage(raw):
    mont1020 = mne.channels.make_standard_montage('standard_1020')
    ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in ALL_CHANNELS]
    mont1020_new = mont1020.copy()
    mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
    kept_channel_info = [mont1020.dig[x+3] for x in ind]
    mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info
    raw.set_montage(mont1020_new)
    return raw

Calculate absolute power

In [ ]:
def calculate_absolute_power(raw_data):
    FMIN=0.01
    FMAX=45
    psds, freqs = mne.time_frequency.psd_array_welch(raw_data.get_data(), fmin=FMIN, fmax=FMAX, sfreq=SFREQ)
    absolute_powers = {}
    for band, (FMIN, FMAX) in FREQ_BANDS.items():
        idx_band = np.logical_and(freqs >= FMIN, freqs <= FMAX)
        absolute_power = np.trapz(psds[:, idx_band], dx=(freqs[1] - freqs[0]), axis=-1)
        absolute_powers[band] = absolute_power
    total_absolute_power = sum(absolute_powers.values())
    return total_absolute_power


#### Time amplitude plot

In [ ]:
def time_amplitude(raw, title):
    fig = raw.plot(
        n_channels=32, 
        scalings=SCALINGS,
        show=SHOW
        )
    fig.savefig(f'MNE-graphs/time-amplitude/{title}-EEG.png')

    print(raw.info)
    # TODO: Extract statistical features from time domain such as mean, median, variance, skewness, kurtosis

#### Power spectral density plot

In [ ]:
def psd(raw, title):
    fig = raw.plot_psd(
        picks=raw.info['ch_names'], 
        show=SHOW)
    fig.savefig(f'MNE-graphs/psd-frequency/{title}.png')

#### Wavelet plot

In [ ]:
# def wavelet(raw1, raw2=None, title):
#     raw1_avg = np.mean(raw1.get_data(), axis=0)
#     t = np.arange(0, 150, 1/SFREQ)
#     ts = t[:-1]
#     wavelet = 'db13'
#     level = 5 # level of decomposition based on your signal characteristics

#     # coeffs_multi_channel = []
#     # for i in range(32): 
#     coeffs1 = pywt.wavedec(raw1_avg, wavelet, level=level)
#         # coeffs_multi_channel.append(coeffs)
    
#     plt.figure(figsize=(10, 6))

#     # raw1
#     plt.subplot(4, 1, 1)
#     plt.plot(ts, raw1_avg, label='xyz')
#     plt.title('Channel 1')
#     plt.xlabel('Time')
#     plt.ylabel('Amplitude')
#     plt.legend()

#     # raw1 coeffs
#     plt.subplot(4, 1, 2)
#     for i in range(level+1):
#         plt.plot(t, pywt.upcoef('a', coeffs1[i], wavelet, level=level)[:len(t)], label=f'Level {i}')
#     plt.title('DWT')
#     plt.xlabel('Time')
#     plt.ylabel('Amplitude')
#     plt.legend()

#     if raw2:
#         raw2_avg = np.mean(raw2.get_data(), axis=0)
#         coeffs2 = pywt.wavedec(raw2_avg, wavelet, level=level)

#         # raw2
#         plt.subplot(4, 1, 3)
#         plt.plot(t, raw2_avg, label='ddd')
#         plt.title('Channel 1')
#         plt.xlabel('Time')
#         plt.ylabel('Amplitude')
#         plt.legend()

#         # raw2 coeffs
#         plt.subplot(4, 1, 4)
#         for i in range(level+1):
#             plt.plot(t, pywt.upcoef('a', coeffs2[i], wavelet, level=level)[:len(t)], label=f'Level {i}')
#         plt.title('DWT')
#         plt.xlabel('Time')
#         plt.ylabel('Amplitude')
#         plt.legend()

#     plt.tight_layout()
#     plt.show()